In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_parquet("../data/safety-SanFrancisco.parquet")
display(data.head())
data.info()

,requestId,dataType,dataSubtype,dateTime,category,subcategory,status,address,latitude,longitude,source,extendedProperties
0,1,Safety,911_Fire,2018-10-02 11:54:01,Potentially Life-Threatening,Medical Incident,None,ELM ST/FRANKLIN ST,37.781286,-122.422187,None,
1,2,Safety,311_All,2018-07-08 15:00:27,Street and Sidewalk Cleaning,Bulky Items,Closed,"1536 SACRAMENTO ST, SAN FRANCISCO, CA, 94109",37.791887,-122.418188,Mobile/Open311,
2,3,Safety,311_All,2016-06-28 13:12:28,General Request - COUNTY CLERK,customer_callback,Closed,Not associated with a specific address,0.000000,0.000000,Phone,
3,7,Safety,311_All,2017-03-03 09:34:49,Temporary Sign Request,Temporary Sign Request for Other_Event_Type,Closed,"2190 NORTH POINT ST, SAN FRANCISCO, CA, 94123",37.802853,-122.443245,Phone,
4,10,Safety,911_Fire,2016-10-25 18:33:20,Potentially Life-Threatening,Medical Incident,None,200 Block of DALEWOOD WAY,37.737953,-122.456498,None,


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3958889 entries, 0 to 3958888
Data columns (total 12 columns):
 #   Column              Dtype  
---  ------              -----  
 0   requestId           int64  
 1   dataType            object 
 2   dataSubtype         object 
 3   dateTime            object 
 4   category            object 
 5   subcategory         object 
 6   status              object 
 7   address             object 
 8   latitude            float64
 9   longitude           float64
 10  source              object 
 11  extendedProperties  object 
dtypes: float64(2), int64(1), object(9)
memory usage: 362.4+ MB


### Checking for Data Quality Issues

### Data Prepping/Cleaning

In [3]:
# looking for 0s, null values, and empty strings

# convert date column to correct type
data['dateTime'] = pd.to_datetime(data['dateTime'])

# change object type to categorical type to save space
data = data.astype({"category": "category", "status": "category", "source": "category", "dataSubtype": "category"})

# convert empty strings to None type
data = data.replace('', np.nan, regex=True)

display(data.isna().sum())

# drop extendedProperties since all values are null
data.drop("extendedProperties", axis=1, inplace=True)

# drop dataType since all values are the same
data.drop("dataType", axis=1, inplace=True)

# drop duplicate rows
data.drop_duplicates(inplace=True, ignore_index=True)

# filter out rows where lat and long are zero
safety_data = data.query("latitude != 0 and longitude != 0")

safety_data.reset_index(drop=True, inplace=True)
print(safety_data.info())
safety_data.head()

requestId                   0
dataType                    0
dataSubtype                 0
dateTime                    0
category                 4933
subcategory                 0
status                1405089
address                     3
latitude                    0
longitude                   0
source                1405089
extendedProperties    3958889
dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3849289 entries, 0 to 3849288
Data columns (total 10 columns):
 #   Column       Dtype         
---  ------       -----         
 0   requestId    int64         
 1   dataSubtype  category      
 2   dateTime     datetime64[ns]
 3   category     category      
 4   subcategory  object        
 5   status       category      
 6   address      object        
 7   latitude     float64       
 8   longitude    float64       
 9   source       category      
dtypes: category(4), datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 190.9+ MB
None


,requestId,dataSubtype,dateTime,category,subcategory,status,address,latitude,longitude,source
0,1,911_Fire,2018-10-02 11:54:01,Potentially Life-Threatening,Medical Incident,NaN,ELM ST/FRANKLIN ST,37.781286,-122.422187,NaN
1,2,311_All,2018-07-08 15:00:27,Street and Sidewalk Cleaning,Bulky Items,Closed,"1536 SACRAMENTO ST, SAN FRANCISCO, CA, 94109",37.791887,-122.418188,Mobile/Open311
2,7,311_All,2017-03-03 09:34:49,Temporary Sign Request,Temporary Sign Request for Other_Event_Type,Closed,"2190 NORTH POINT ST, SAN FRANCISCO, CA, 94123",37.802853,-122.443245,Phone
3,10,911_Fire,2016-10-25 18:33:20,Potentially Life-Threatening,Medical Incident,NaN,200 Block of DALEWOOD WAY,37.737953,-122.456498,NaN
4,12,311_All,2020-04-10 12:45:00,Street and Sidewalk Cleaning,Medical Waste,Closed,"4 TURK ST, SAN FRANCISCO, CA, 94102",37.783520,-122.409487,Web


In [4]:
display(safety_data.isna().sum())

# get % of missing data
category_percentage = (safety_data["category"].isna().sum() / len(safety_data)) * 100
status_percentage = (safety_data["status"].isna().sum() / len(safety_data)) * 100
source_percentage = (safety_data["source"].isna().sum() / len(safety_data)) * 100

print(f"% of missing data for category: {category_percentage:.4f}%")
print(f"% of missing data for status: {status_percentage:.4f}%")
print(f"% of missing data for source: {source_percentage:.4f}%")

requestId            0
dataSubtype          0
dateTime             0
category          4933
subcategory          0
status         1405088
address              3
latitude             0
longitude            0
source         1405088
dtype: int64

% of missing data for category: 0.1282%
% of missing data for status: 36.5025%
% of missing data for source: 36.5025%


In [5]:
print("----Source----")
display(safety_data["source"].value_counts())

print("\n\n----Status----")
display(safety_data["status"].value_counts())

----Source----


Mobile/Open311       1339613
Phone                 732194
Web                   289876
Integrated Agency      69142
Twitter                12031
Other Department        1337
Email                      8
Mail                       0
Name: source, dtype: int64



----Status----


Closed    2418246
Open        25955
Name: status, dtype: int64

In [6]:
print(safety_data["status"].isna().sum())

check = safety_data.loc[safety_data["status"].isna()]
print(check["source"].isna().sum())

# it looks like rows that have a null status also have a null source

1405088
1405088


In [7]:
(safety_data["source"].isna().sum() / len(safety_data)) * 100

# we can't simply drop those rows because we'd lose > 36% of our data!

36.50253332498547

In [8]:
# I'll go ahead and drop the missing data from the category column
cleaned_data = safety_data.loc[~safety_data["category"].isna()]
cleaned_data.reset_index(drop=True, inplace=True)
print(cleaned_data.info())
cleaned_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3844356 entries, 0 to 3844355
Data columns (total 10 columns):
 #   Column       Dtype         
---  ------       -----         
 0   requestId    int64         
 1   dataSubtype  category      
 2   dateTime     datetime64[ns]
 3   category     category      
 4   subcategory  object        
 5   status       category      
 6   address      object        
 7   latitude     float64       
 8   longitude    float64       
 9   source       category      
dtypes: category(4), datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 190.6+ MB
None


,requestId,dataSubtype,dateTime,category,subcategory,status,address,latitude,longitude,source
0,1,911_Fire,2018-10-02 11:54:01,Potentially Life-Threatening,Medical Incident,NaN,ELM ST/FRANKLIN ST,37.781286,-122.422187,NaN
1,2,311_All,2018-07-08 15:00:27,Street and Sidewalk Cleaning,Bulky Items,Closed,"1536 SACRAMENTO ST, SAN FRANCISCO, CA, 94109",37.791887,-122.418188,Mobile/Open311
2,7,311_All,2017-03-03 09:34:49,Temporary Sign Request,Temporary Sign Request for Other_Event_Type,Closed,"2190 NORTH POINT ST, SAN FRANCISCO, CA, 94123",37.802853,-122.443245,Phone
3,10,911_Fire,2016-10-25 18:33:20,Potentially Life-Threatening,Medical Incident,NaN,200 Block of DALEWOOD WAY,37.737953,-122.456498,NaN
4,12,311_All,2020-04-10 12:45:00,Street and Sidewalk Cleaning,Medical Waste,Closed,"4 TURK ST, SAN FRANCISCO, CA, 94102",37.783520,-122.409487,Web


In [9]:
# save parquet file
cleaned_data.to_parquet("../data/cleanedSanFranData.parquet")